In [ ]:
import pandas as pd
import sympy as sp
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import subprocess
import signal
import os

from glob import glob


import gmsh
from enum import Enum, auto
from mpi4py import MPI
from scipy import interpolate

In [ ]:
current_ratio_U238_to_U235 = 137.818

half_life_U235 = 703.8e6  # Half-life of U-235 in years
half_life_U238 = 4.468e9  # Half-life of U-238 in years


current_U238_fraction = 0.992745
current_U235_fraction = 0.00720
current_ratio_U238_to_U235 = 137.818 ### from isoplotR


# Decay constant of U238
lambda_U238 = np.log(2) / half_life_U238
lambda_U235 = np.log(2) / half_life_U235

In [ ]:
lambda_U238,  lambda_U235

In [ ]:
def concordia_curve(t, lambda_235=9.8485e-10, lambda_238=1.55125e-10):
    
    current_ratio_U238_to_U235 = 137.818 ### from isoplotR
    # Calculate the exponential growth for both decay systems
    e_lambda_235t = np.exp(lambda_235 * t)
    e_lambda_238t = np.exp(lambda_238 * t)
    
    # Calculate ratios
    Pb206_U238 = e_lambda_238t - 1
    Pb207_U235 = e_lambda_235t - 1
    Pb207_Pb206 = (Pb207_U235 / Pb206_U238) * (1 / current_ratio_U238_to_U235)  # Natural ratio of U235/U238

    return Pb207_Pb206, Pb206_U238

#### Extract the analytical data

In [ ]:
TB14_093 = pd.read_csv('./TB14_093.csv')

In [ ]:
Ti_data = TB14_093['Ti49_ppm_mean']

In [ ]:
# Define the symbols
T = sp.symbols('T')
T_i = sp.symbols('T_i')
C_Ti = sp.symbols('C_Ti')
ASi = sp.symbols('aSiO_2')
ATi = sp.symbols('aTiO_2')
# ASi = 1  # Set SiO2 activity
# ATi = 0.7  # Set TiO2 activity

# Create the expression for C_i
Ti_C_expr = 10**(5.711 - (4800 / (T + 273)) - sp.log(ASi, 10) + sp.log(ATi, 10))

# Ti_C_fn = lambdify(T_i, Ti_C_expr, 'numpy')

T_expr = 4800 / (5.711 - sp.log(C_Ti, 10) - sp.log(ASi, 10) + sp.log(ATi, 10)) - 273

# Ti_T_fn = sp.lambdify(C_Ti, T_expr, 'numpy')

In [ ]:
Ti_T_fn_07 = sp.lambdify(C_Ti, T_expr.subs({ASi:1, ATi:0.7}), 'numpy')
Ti_T_fn_08 = sp.lambdify(C_Ti, T_expr.subs({ASi:1, ATi:0.8}), 'numpy')
Ti_T_fn_09 = sp.lambdify(C_Ti, T_expr.subs({ASi:1, ATi:0.9}), 'numpy')
Ti_T_fn_1 = sp.lambdify(C_Ti, T_expr.subs({ASi:1, ATi:1}), 'numpy')

In [ ]:
# for i in [Ti_T_fn_07, Ti_T_fn_08, Ti_T_fn_09, Ti_T_fn_1]:
sns.kdeplot(data=Ti_T_fn_07(Ti_data), label=r'$\alpha$SiO$_2$=1, $\alpha$TiO$_2$=0.7')
sns.kdeplot(data=Ti_T_fn_08(Ti_data), label=r'$\alpha$SiO$_2$=1, $\alpha$TiO$_2$=0.8')
sns.kdeplot(data=Ti_T_fn_09(Ti_data), label=r'$\alpha$SiO$_2$=1, $\alpha$TiO$_2$=0.9')
sns.kdeplot(data=Ti_T_fn_1(Ti_data), label=r'$\alpha$SiO$_2$=1, $\alpha$TiO$_2$=1.0')

# plt.text(1000, 0.0075, r'$\alpha$SiO$_2$=1')

plt.xlabel('T from Ti [$\degree$C]')

plt.legend()

plt.savefig('Temp_from_Ti_TB-14_093.pdf')

In [ ]:
# Time range in years
time = np.linspace(200e6, 2100e6, 1000)  # 0 to 1000 Ma

time1 = np.linspace(500e6, 2000e6, 16)

# time2 = np.linspace(550e6, 1050e6, 6)



# Generate data for the Concordia curve
pb207_pb206, pb206_u238 = concordia_curve(time)

pb207_pb206_1, pb206_u238_1 = concordia_curve(time1)


# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(1/pb206_u238_1, pb207_pb206_1, c='orange', label='time marker', s=200)
plt.plot(1/pb206_u238, pb207_pb206, label='Concordia Curve', c='k')
for i in range(len(pb206_u238_1)):
    plt.text( (1/pb206_u238_1[i])-0.1, pb207_pb206_1[i]+0.0002, f'{round(time1[i]/1e6)} Ma')

temp_estimate = Ti_T_fn_07(Ti_data)

Tcbar = plt.scatter(TB14_093['Final U238/Pb206_mean'][temp_estimate < 950], TB14_093['Final Pb207/Pb206_mean'][temp_estimate < 950], c=temp_estimate[temp_estimate < 950], vmin=750, vmax=925)
cbar = plt.colorbar(Tcbar, extend='both')
cbar.set_label('T [$\degree$C]')

plt.ylim(0.05, 0.125)
plt.xlim(2, 14)

plt.plot([1/concordia_curve(1950e6)[1], 1/concordia_curve(470e6)[1]], [concordia_curve(1950e6)[0], concordia_curve(470e6)[0]], c='k', ls=':')
plt.plot([1/concordia_curve(1950e6)[1], 1/concordia_curve(590e6)[1]], [concordia_curve(1950e6)[0], concordia_curve(590e6)[0]], c='k', ls=':')

plt.savefig('TW-UPb_TB-14_093.pdf')



#### Generate PTt paths for testing

In [ ]:
import scipy 

x_points = [0, 40, 75, 90, 100, 125, 200, 1340, 1350, 1380, 1395, 1410, 1440, 1460, 1950]


y_points = [800, 790, 780, 770, 760, 725, 725, 725, 825, 850, 850, 850, 825, 725, 725]

# Generate x values for plotting
x_values = np.linspace(0, 1950, 500)
y_interp_850 = scipy.interpolate.interp1d(x_points, y_points, kind='slinear')
y_850 = y_interp_850(x_values)


y_points = [800, 790, 780, 770, 760, 725, 725, 725, 850, 900, 900, 900, 850, 725, 725]


import scipy# Generate x values for plotting
x_values = np.linspace(0, 1950, 500)
y_interp_900 = scipy.interpolate.interp1d(x_points, y_points, kind='slinear')
y_900 = y_interp_900(x_values)


y_points = [800, 790, 780, 770, 760, 725, 725, 725, 900, 950, 950, 950, 900, 725, 725]
y_interp_975 = scipy.interpolate.interp1d(x_points, y_points, kind='slinear')
y_950 = y_interp_975(x_values)



y_points = [800, 790, 780, 770, 760, 725, 725, 725, 900, 1000, 1000, 1000, 900, 725, 725]
y_interp_1000 = scipy.interpolate.interp1d(x_points, y_points, kind='slinear')
y_1000 = y_interp_1000(x_values)



plt.xlabel('Time [Ma]')
plt.ylabel('Temperature [$\degree$C]')


plt.plot([600, 600], [500, 1100], c='k', ls='--')

# plt.plot([580, 580], [500, 1100], c='k', ls=':')

plt.plot([510, 510], [500, 1100], c='k', ls=':')

plt.plot([555, 555], [500, 1100], c='k', ls='-.')



plt.plot( abs(x_values-1950), scipy.ndimage.gaussian_filter(y_850, 2), label='profile 1' )
plt.plot( abs(x_values-1950), scipy.ndimage.gaussian_filter(y_900, 2), label='profile 2'  )
plt.plot( abs(x_values-1950), scipy.ndimage.gaussian_filter(y_950, 2), label='profile 3'  )
plt.plot( abs(x_values-1950), scipy.ndimage.gaussian_filter(y_1000, 2), label='profile 4'  )


plt.xlim(1950, 0)

plt.ylim(550, 1100)

# plt.plot([1480, 1480], [500, 1100])

# plt.plot([460, 460], [500, 1100], c='k', ls=':')


plt.legend(loc='upper left')

plt.grid(ls=':', alpha=0.5)


plt.savefig(f'./figs/TB_comparison_Tt_profiles_full_Tt.pdf')

#### Create meshes for UW

In [ ]:
# Initialize the MPI environment
comm = MPI.COMM_WORLD  # Get the global communicator
rank = comm.Get_rank()  # Get the rank of the current process
size = comm.Get_size()  # Get the total number of processes

In [ ]:
# Create a dynamic Enum class based on the number of points
def create_boundary_enum(num_points):
    """Dynamically create a boundary Enum class."""
    enum_dict = {f"Boundary{i}": auto() for i in range(num_points)}
    return Enum("Boundaries", enum_dict)

In [ ]:
def create_2D_zircon_mesh(points, outputPath='.', csize = 0.01, mesh_name = 'zircon_mesh'):

    if rank == 0:
        points_array = np.array(points)
    
        # Generate the Enum class
        boundaries_2D = create_boundary_enum(len(points))
        
        gmsh.initialize()
        gmsh.model.add(mesh_name)
        
        
        # Create points
        point_ids = [gmsh.model.geo.addPoint(x, y, z, meshSize=csize) for x, y, z in points]
        
        # Create lines by connecting consecutive points and closing the loop
        line_ids = [gmsh.model.geo.addLine(point_ids[i], point_ids[(i + 1) % len(point_ids)]) for i in range(len(point_ids))]
        
        cl = gmsh.model.geo.addCurveLoop(line_ids)
        surface = gmsh.model.geo.addPlaneSurface([cl])
        
        gmsh.model.geo.synchronize()
        
        # Adding physical groups for lines
        for i, line_id in enumerate(line_ids):
            boundary_tag = getattr(boundaries_2D, f"Boundary{i}")
            gmsh.model.addPhysicalGroup(1, [line_id], tag=boundary_tag.value, name=boundary_tag.name)
        
        # Add physical group for the surface
        gmsh.model.addPhysicalGroup(2, [surface], 99999)
        gmsh.model.setPhysicalName(2, 99999, "Elements")
        
        gmsh.model.mesh.generate(2)
        gmsh.write(f'{outputPath}/{mesh_name}.msh')
        
        gmsh.finalize()

        print(f'mesh located can be found: {outputPath}/{mesh_name}.msh', flush=True)

In [ ]:
ref_length = 100 ### in microns


mesh_output = './meshes/'

fig_output = './figs/'

model_output = './model_output/'

os.makedirs(mesh_output, exist_ok=True)
os.makedirs(fig_output, exist_ok=True)
os.makedirs(model_output, exist_ok=True)


In [ ]:
### 70 x 70 micron zircon

zircon1 = [(-35, 20, 0), (-20, 35, 0), (20, 35, 0), (35, 20, 0), 
           (35, -20, 0), (20, -35, 0), (-20, -35, 0), (-35, -20, 0)]

zircon_70_70 = np.array(zircon1)

create_2D_zircon_mesh(zircon_70_70/ref_length, outputPath=mesh_output, csize = 0.01, mesh_name = 'zircon_mesh_70x70')

In [ ]:
### 80x120 micron zircon

zircon2 = [(-40, 35, 0), (-25, 60, 0), (25, 60, 0), (40, 35, 0), 
           (40, -35, 0), (25, -60, 0), (-25, -60, 0), (-40, -35, 0)]

zircon_80_120 = np.array(zircon2)

create_2D_zircon_mesh(zircon_80_120/ref_length, outputPath=mesh_output, csize = 0.01, mesh_name = 'zircon_mesh_80x120')

In [ ]:
### 90x160 micron zircon

zircon3 = [(-45, 40, 0), (-30, 80, 0), (30, 80, 0), (45, 40, 0), 
           (45, -40, 0), (30, -80, 0), (-30, -80, 0), (-45, -40, 0)]

zircon_90_160 = np.array(zircon3)

create_2D_zircon_mesh(zircon_90_160/ref_length, outputPath=mesh_output, csize = 0.01, mesh_name = 'zircon_mesh_90x160')

In [ ]:
### 100x200 micron zircon

zircon4 = [(-50, 50, 0), (-35, 100, 0), (35, 100, 0), (50, 50, 0), 
           (50, -50, 0), (35, -100, 0), (-35, -100, 0), (-50, -50, 0)]

zircon_100_200 = np.array(zircon4)

create_2D_zircon_mesh(zircon_100_200/ref_length, outputPath=mesh_output, csize = 0.01, mesh_name = 'zircon_mesh_100x200')

In [ ]:
plt.figure( figsize=(4, 8) )
plt.plot(zircon_70_70[:,0], zircon_70_70[:,1], label='zircon 1')
plt.plot(zircon_80_120[:,0], zircon_80_120[:,1], label='zircon 2')
plt.plot(zircon_90_160[:,0], zircon_90_160[:,1], label='zircon 3')
plt.plot(zircon_100_200[:,0], zircon_100_200[:,1], label='zircon 4')

plt.axis('scaled')
plt.grid(alpha=0.4, ls=':')

plt.xlabel(r'x [$\mu m$]')
plt.ylabel(r'y [$\mu m$]')

plt.savefig(f'{fig_output}mesh_geom.pdf')

#### Functions for sampling the numerical models

In [ ]:
from matplotlib import pyplot as plt, patches
import matplotlib.path as mpltPath

def create_sample_spot(centre, radius):

    ### create a circle at centre of box to represent garnet
    circle = patches.Circle(centre, radius)
    
    verts = circle.get_path().vertices
    trans = circle.get_patch_transform()
    circlePoints = trans.transform(verts)
    circleShape = mpltPath.Path(circlePoints)

    return circleShape

def sample_spot(spot, isotope, mesh):
    # with mesh.access(isotope):
    #     data = isotope.data[spot.contains_points(isotope.coords[:])] 
    # data = mesh[isotope][spot.contains_points(mesh.points[:])]
    data = mesh[isotope][spot.contains_points(mesh.points[:,0:2])]
    #     data = uw.utilities.gather_data(data, bcast=True)

    spot_average_data = np.average(data)


    return spot_average_data

def extract_spot_data(directory, centre_points, sample_r):
    ''' 
    order of array
    Pb206_Pb206, Pb207_Pb207, U235_U235, U238_U238
    '''
    mesh = pv.read(natsorted(glob(f'{directory}*swarm.fields.*.xdmf'))[-1])

    try:
        mesh.point_data.remove('time')
    except:
        pass
    
    spot_data = np.zeros(shape=(centre_points.shape[0], 4))
    
    i = 0
    for centre in centre_points:
        spot = create_sample_spot(centre, sample_r)
        x = 0
        for name in mesh.array_names:
            data = sample_spot(spot, name, mesh)
            spot_data[i, x] = data
            x+=1
    
        i += 1

    return spot_data

def extract_point_data(directory, points):
    mesh_data = pv.read(natsorted(glob(f'{directory}*step*xdmf'))[-1])

    spot_data = pv.PolyData(points).sample(mesh_data)

    return spot_data

### Models need to be restarted after ~24 hours
This function restarts them automatically

In [ ]:
def run_model(profile_name, mesh, start_time, time_str, temp_str, diffusion_type='slow'):
    '''
    profile_name [str]: name of profile being tested
    mesh [str]: name of mesh to be used
    start_time [float]: time to start model along temp-time path
    time_str [str]: string of values to pass to UW for the model time along the Tt path
    temp_str [str]: string of values to pass to UW for the model temp along the Tt path
    timeout [float]: value at which to restart the model (they need restarting every 24 hours) in seconds
    diffusion_type [str]: string of 'fast' or 'slow' diffusion to replicate radiation damage (default to slow)
    
    '''

    mesh_file = f'{mesh_output}{mesh}.msh'
    csize = 0.02
    udegree = 1

    file_name = f'{profile_name}-{diffusion_type}-{mesh}_diffusion-start_time={start_time}-deg={udegree}-csize={csize}'
    nruns = len(glob(f'{model_output}*{file_name}*.txt'))
    output_path = f'{model_output}{file_name}/'
    std_out = f'{model_output}{file_name}_stdout-{nruns}.txt'
    

    # command to be passed as a list
    command = [
        "nohup", "python3", "TB_zircon_model_UW.py",
        "-uw_csize", str(csize),
        "-uw_degree", str(udegree),
        "-uw_mesh_file", mesh_file,
        "-uw_output_path", output_path,
        "-uw_time_arr", time_str,
        "-uw_temp_arr", temp_str,
        "-uw_diffusion_type", diffusion_type
    ]

    # os.makedirs(output_path, exist_ok=True)
    
    # Open the stdout file to write logs
    with open(f'{std_out}', 'w') as outfile:
        # Run the command detached from the terminal
        process = subprocess.Popen(
            command,
            stdout=outfile,
            stderr=subprocess.STDOUT,
            preexec_fn=os.setpgrp  # Detaches the process to run in its own process group
        )

    process_pid = process.pid

    print(f"Started {mesh} {profile_name} with PID: {process_pid}")

    return process
            
    
    

#### Loop to restart the model automatically

In [ ]:
mesh_list = ['zircon_mesh_70x70', 'zircon_mesh_80x120', 'zircon_mesh_90x160', 'zircon_mesh_100x200']

In [ ]:
import time
nchecks = 20   # how many checks to perform each half hour to see if model is still running

finished = False

max_restarts = 5

time_str = ','.join(map(str, list( x_values  )))



# temp_str = ','.join(map(str, list(scipy.ndimage.gaussian_filter(y_850, 2))))
# profile_name = 'TB_1950Ma_850C_peak_at_555Ma'

# temp_str = ','.join(map(str, list(scipy.ndimage.gaussian_filter(y_900, 2))))
# profile_name = 'TB_1950Ma_900C_peak_at_555Ma'

# temp_str = ','.join(map(str, list(scipy.ndimage.gaussian_filter(y_950, 2))))
# profile_name = 'TB_1950Ma_950C_peak_at_555Ma'

temp_str = ','.join(map(str, list(scipy.ndimage.gaussian_filter(y_1000, 2))))
profile_name = 'TB_1950Ma_1000C_peak_at_555Ma'


running_dict = dict.fromkeys(mesh_list, None)

for i in range(max_restarts):
    ### start each model for different mesh
    for key, value in list( running_dict.items() ):
        if value != None:
            print(f'killing {key}, process ID: {value.pid}')
            os.kill(int(value.pid), signal.SIGKILL)

        print(f'{key} number of restarts: {i}')
        ### run the model in background and get PID
        process_details = run_model(profile_name, key, 0, time_str, temp_str, 'slow')
        running_dict[key] = process_details
        time.sleep(10) ### 10 seconds wait between starting each job
        if process_details.poll() != None:
            ### check if model is running, if not remove mesh from the dictionary 
            print(f'{key} has finished')
            running_dict.pop(key)
            

    ### check if process is running every hour
    for t in range(0, nchecks):
        print(len(running_dict))
        time.sleep(30*60) ### 30 mins (1800 seconds) between checking if models are still running
        for key, value in list( running_dict.items() ):
            ### remove from dict if it's finished
            if value.poll() != None:
                print(f'{key} has finished')
                running_dict.pop(key)
        
        if len(running_dict) == 0:
            print('all models finished')
            break

    for key, value in list( running_dict.items() ):
        if value.poll() != None:
            print(f'{key} has finished')
            running_dict.pop(key)

    if len(running_dict) == 0:
        print('all models finished')
        break

    ### restart after 10 seconds
    time.sleep(10)

#### plot the model output and compare with analytical data

In [ ]:
sample_r = 0.23 / 2 ### 23 micron diameter


### Set a seed for reproducibility
np.random.seed(0)

### number of spots from zircon
nsamples = 100


mesh_list = ['zircon_mesh_70x70', 'zircon_mesh_80x120', 'zircon_mesh_90x160', 'zircon_mesh_100x200']

In [ ]:
# Generate data for the Concordia curve
time_det = np.linspace(200e6, 2100e6, 1000)  # 0 to 1000 Ma

pb207_pb206, pb206_u238 = concordia_curve(time_det)

pb207_pb206_1, pb206_u238_1 = concordia_curve(time1)


In [ ]:
import pyvista as pv
from natsort import natsorted

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
from natsort import natsorted
from glob import glob

def plot_profile_data(profile_dirs, sample_r, nsamples,
                      pb206_u238, pb207_pb206,
                      pb206_u238_1, pb207_pb206_1,
                      time1, TB14_093, temp_estimate, mesh_list):
    """
    Plots the concordia and spot data from the profile directories.

    Parameters:
        profile_dirs (list): List of directory paths containing model output.
        sample_r (float): Sample radius to use when centering spot data.
        nsamples (int): Number of samples to generate for each profile.
        pb206_u238 (array-like): Array used for the concordia curve x-data.
        pb207_pb206 (array-like): Array used for the concordia curve y-data.
        pb206_u238_1 (array-like): Array for the time marker x-values.
        pb207_pb206_1 (array-like): Array for the time marker y-values.
        time1 (array-like): Array of times (used for adding text labels).
        TB14_093 (dict or DataFrame): Contains keys 'Approx_U_PPM_mean', 'Final U238/Pb206_mean', and 'Final Pb207/Pb206_mean'.
        temp_estimate (array-like): Temperature estimates used to filter TB14_093 data.
        mesh_list (list): List of mesh names (used to generate labels).
    """
    plt.figure(figsize=(10, 6))
    # Plot the time marker scatter and concordia curve
    plt.scatter(1/np.array(pb206_u238_1), pb207_pb206_1, c='orange', label='time marker', s=200)
    plt.plot(1/np.array(pb206_u238), pb207_pb206, label='Concordia Curve', c='k')
    
    # Loop over each profile directory to extract and plot spot data
    i = 0
    for directory in profile_dirs:
        # Read the mesh from the XDMF file for the current directory
        mesh = pv.read(f'{directory}/timesteps/_step_00000.xdmf')
        x_min, x_max = mesh.points[:,0].min(), mesh.points[:,0].max()
        y_min, y_max = mesh.points[:,1].min(), mesh.points[:,1].max()
        x_lower, x_upper = x_min + (sample_r * 1.1), x_max - (sample_r * 1.1)
        y_lower, y_upper = y_min + (sample_r * 1.1), y_max - (sample_r * 1.1)

        # Generate random sample centre points within the region
        sample_centre_points = np.random.uniform(low=(x_lower, y_lower), 
                                                  high=(x_upper, y_upper), 
                                                  size=(nsamples, 2))
        
        # extract_spot_data is assumed to be defined elsewhere; it should return an array.
        spot_data = extract_spot_data(f'{directory}/timesteps/', sample_centre_points, sample_r)
        
        # Plot each spot's data. The ratios are calculated from spot_data columns.
        plt.scatter(spot_data[:,3] / spot_data[:,0],
                    spot_data[:,1] / spot_data[:,0],
                    marker='v', s=150,
                    label=f'{mesh_list[::-1][i][12:]} $\mu m$',
                    alpha=0.5)
        i += 1

    # Plot TB14-093 data and add a colorbar based on U concentration
    U_ppm = TB14_093['Approx_U_PPM_mean']
    Tcbar = plt.scatter(TB14_093['Final U238/Pb206_mean'][temp_estimate < 950],
                        TB14_093['Final Pb207/Pb206_mean'][temp_estimate < 950],
                        c=U_ppm[temp_estimate < 950], vmax=2500,
                        label='TB14-093 data')
    cbar = plt.colorbar(Tcbar, extend='both')
    cbar.set_label('U [ppm]')
    
    plt.legend()

    # Annotate each time marker with a time in Ma (converted from time1)
    for i in range(len(pb206_u238_1)):
        plt.text((1/np.array(pb206_u238_1)[i]) - 0.1, 
                 pb207_pb206_1[i] + 0.0002, 
                 f'{round(time1[i]/1e6)} Ma')
    
    plt.xlim(2.5, 14.5)
    plt.ylim(0.045, 0.13)
    plt.savefig('./figs/{fileName}.pdf')
    plt.show()

In [ ]:
profile_dirs = natsorted(glob('./model_output/*850C_peak*'))[::-1][:]
plot_profile_data(profile_dirs, sample_r, nsamples, pb206_u238, pb207_pb206, pb206_u238_1, pb207_pb206_1, time1, TB14_093, temp_estimate, mesh_list, fileName='TB_1950Ma_850C_peak_at_555Ma')

In [ ]:
profile_dirs = natsorted(glob('./model_output/*900C_peak*'))[::-1][:]
plot_profile_data(profile_dirs, sample_r, nsamples, pb206_u238, pb207_pb206, pb206_u238_1, pb207_pb206_1, time1, TB14_093, temp_estimate, mesh_list, fileName='TB_1950Ma_900C_peak_at_555Ma')

In [ ]:
profile_dirs = natsorted(glob('./model_output/*950C_peak*'))[::-1][:]
plot_profile_data(profile_dirs, sample_r, nsamples, pb206_u238, pb207_pb206, pb206_u238_1, pb207_pb206_1, time1, TB14_093, temp_estimate, mesh_list, fileName='TB_1950Ma_950C_peak_at_555Ma')

In [ ]:
profile_dirs = natsorted(glob('./model_output/*1000C_peak*'))[::-1][:]
plot_profile_data(profile_dirs, sample_r, nsamples, pb206_u238, pb207_pb206, pb206_u238_1, pb207_pb206_1, time1, TB14_093, temp_estimate, mesh_list, fileName='TB_1950Ma_1000C_peak_at_555Ma')

In [ ]:
# profile_dirs = natsorted(glob('./model_output/*850C_peak*'))[::-1][:]



# plt.figure(figsize=(10, 6))
# plt.scatter(1/pb206_u238_1, pb207_pb206_1, c='orange', label='time marker', s=200)
# plt.plot(1/pb206_u238, pb207_pb206, label='Concordia Curve', c='k')
# i = 0
# for directory in profile_dirs:
#     mesh = pv.read(f'{directory}/timesteps/_step_00000.xdmf')
#     x_min, x_max = mesh.points[:,0].min(), mesh.points[:,0].max()
#     y_min, y_max = mesh.points[:,1].min(), mesh.points[:,1].max()
#     x_lower, x_upper = x_min+(sample_r*1.1), x_max-(sample_r*1.1)
#     y_lower, y_upper = y_min+(sample_r*1.1), y_max-(sample_r*1.1)
#     sample_centre_points = np.random.uniform(low=(x_lower, y_lower), high=(x_upper, y_upper), size=(nsamples, 2))
    
#     spot_data = extract_spot_data(f'{directory}/timesteps/', sample_centre_points, sample_r)


#     plt.scatter(spot_data[:,3]/spot_data[:,0], spot_data[:,1]/spot_data[:,0], marker='v', s = 150, label=f'{mesh_list[::-1][i][12:]} $\mu m$',alpha=0.5)

#     i += 1

# U_ppm = TB14_093['Approx_U_PPM_mean']

# Tcbar = plt.scatter(TB14_093['Final U238/Pb206_mean'][temp_estimate < 950], TB14_093['Final Pb207/Pb206_mean'][temp_estimate < 950], c=U_ppm[temp_estimate < 950], vmax=2500, label='TB14-093 data')
# cbar = plt.colorbar(Tcbar, extend='both')
# cbar.set_label('U [ppm]')

# plt.legend()

# for i in range(len(pb206_u238_1)):
#     plt.text( (1/pb206_u238_1[i])-0.1, pb207_pb206_1[i]+0.0002, f'{round(time1[i]/1e6)} Ma')





# plt.xlim(2.5, 14.5)
# plt.ylim(0.045, 0.13)

# plt.savefig('./figs/T_max=850C_t_max=555Ma_concordia_plot.pdf')


In [ ]:
# profile_dirs = natsorted(glob('./model_output/*900C_peak*'))[::-1][:]


# plt.figure(figsize=(10, 6))
# plt.scatter(1/pb206_u238_1, pb207_pb206_1, c='orange', label='time marker', s=200)
# plt.plot(1/pb206_u238, pb207_pb206, label='Concordia Curve', c='k')
# i = 0
# for directory in profile_dirs:
#     mesh = pv.read(f'{directory}/timesteps/_step_00000.xdmf')
#     x_min, x_max = mesh.points[:,0].min(), mesh.points[:,0].max()
#     y_min, y_max = mesh.points[:,1].min(), mesh.points[:,1].max()
#     x_lower, x_upper = x_min+(sample_r*1.1), x_max-(sample_r*1.1)
#     y_lower, y_upper = y_min+(sample_r*1.1), y_max-(sample_r*1.1)
#     sample_centre_points = np.random.uniform(low=(x_lower, y_lower), high=(x_upper, y_upper), size=(nsamples, 2))
    
#     spot_data = extract_spot_data(f'{directory}/timesteps/', sample_centre_points, sample_r)


#     plt.scatter(spot_data[:,3]/spot_data[:,0], spot_data[:,1]/spot_data[:,0], marker='v', s = 150, label=f'{mesh_list[::-1][i][12:]} $\mu m$',alpha=0.5)

#     i += 1

# U_ppm = TB14_093['Approx_U_PPM_mean']

# Tcbar = plt.scatter(TB14_093['Final U238/Pb206_mean'][temp_estimate < 950], TB14_093['Final Pb207/Pb206_mean'][temp_estimate < 950], c=U_ppm[temp_estimate < 950], vmax=2500, label='TB14-093 data')
# cbar = plt.colorbar(Tcbar, extend='both')
# cbar.set_label('U [ppm]')

# plt.legend()

# for i in range(len(pb206_u238_1)):
#     plt.text( (1/pb206_u238_1[i])-0.1, pb207_pb206_1[i]+0.0002, f'{round(time1[i]/1e6)} Ma')





# plt.xlim(2.5, 14.5)
# plt.ylim(0.045, 0.13)

# plt.savefig('./figs/T_max=900C_t_max=555Ma_concordia_plot.pdf')


In [ ]:
# profile_dirs = natsorted(glob('./model_output/*950C_peak*'))[::-1][:]




# plt.figure(figsize=(10, 6))
# plt.scatter(1/pb206_u238_1, pb207_pb206_1, c='orange', label='time marker', s=200)
# plt.plot(1/pb206_u238, pb207_pb206, label='Concordia Curve', c='k')
# i = 0
# for directory in profile_dirs:
#     print(directory)
#     mesh = pv.read(f'{directory}/timesteps/_step_00000.xdmf')
#     x_min, x_max = mesh.points[:,0].min(), mesh.points[:,0].max()
#     y_min, y_max = mesh.points[:,1].min(), mesh.points[:,1].max()
#     x_lower, x_upper = x_min+(sample_r*1.1), x_max-(sample_r*1.1)
#     y_lower, y_upper = y_min+(sample_r*1.1), y_max-(sample_r*1.1)
#     sample_centre_points = np.random.uniform(low=(x_lower, y_lower), high=(x_upper, y_upper), size=(nsamples, 2))
    
#     spot_data = extract_spot_data(f'{directory}/timesteps/', sample_centre_points, sample_r)


#     plt.scatter(spot_data[:,3]/spot_data[:,0], spot_data[:,1]/spot_data[:,0], marker='v', s = 150, label=f'{mesh_list[::-1][i][12:]} $\mu m$',alpha=0.5)

#     i += 1

# U_ppm = TB14_093['Approx_U_PPM_mean']

# Tcbar = plt.scatter(TB14_093['Final U238/Pb206_mean'][temp_estimate < 950], TB14_093['Final Pb207/Pb206_mean'][temp_estimate < 950], c=U_ppm[temp_estimate < 950], vmax=2500, label='TB14-093 data')
# cbar = plt.colorbar(Tcbar, extend='both')
# cbar.set_label('U [ppm]')

# plt.legend()

# for i in range(len(pb206_u238_1)):
#     plt.text( (1/pb206_u238_1[i])-0.1, pb207_pb206_1[i]+0.0002, f'{round(time1[i]/1e6)} Ma')





# plt.xlim(2.5, 14.5)
# plt.ylim(0.045, 0.13)

# plt.savefig('./figs/T_max=950C_t_max=555Ma_concordia_plot.pdf')


In [ ]:
# profile_dirs = natsorted(glob('./model_output/*1000C_peak*'))[::-1][:]




# plt.figure(figsize=(10, 6))
# plt.scatter(1/pb206_u238_1, pb207_pb206_1, c='orange', label='time marker', s=200)
# plt.plot(1/pb206_u238, pb207_pb206, label='Concordia Curve', c='k')
# i = 0
# for directory in profile_dirs:
#     mesh = pv.read(f'{directory}/timesteps/_step_00000.xdmf')
#     x_min, x_max = mesh.points[:,0].min(), mesh.points[:,0].max()
#     y_min, y_max = mesh.points[:,1].min(), mesh.points[:,1].max()
#     x_lower, x_upper = x_min+(sample_r*1.1), x_max-(sample_r*1.1)
#     y_lower, y_upper = y_min+(sample_r*1.1), y_max-(sample_r*1.1)
#     sample_centre_points = np.random.uniform(low=(x_lower, y_lower), high=(x_upper, y_upper), size=(nsamples, 2))
    
#     spot_data = extract_spot_data(f'{directory}/timesteps/', sample_centre_points, sample_r)


#     plt.scatter(spot_data[:,3]/spot_data[:,0], spot_data[:,1]/spot_data[:,0], marker='v', s = 150, label=f'{mesh_list[::-1][i][12:]} $\mu m$',alpha=0.5)

#     i += 1

# U_ppm = TB14_093['Approx_U_PPM_mean']

# Tcbar = plt.scatter(TB14_093['Final U238/Pb206_mean'][temp_estimate < 950], TB14_093['Final Pb207/Pb206_mean'][temp_estimate < 950], c=U_ppm[temp_estimate < 950], vmax=2500, label='TB14-093 data')
# cbar = plt.colorbar(Tcbar, extend='both')
# cbar.set_label('U [ppm]')

# plt.legend()

# for i in range(len(pb206_u238_1)):
#     plt.text( (1/pb206_u238_1[i])-0.1, pb207_pb206_1[i]+0.0002, f'{round(time1[i]/1e6)} Ma')





# plt.xlim(2.5, 14.5)
# plt.ylim(0.045, 0.13)

# plt.savefig('./figs/T_max=1000C_t_max=555Ma_concordia_plot.pdf')
